En este notebook realizaremos un problema de regresión donde calcularemos el valor de la variable ``trueE`` ya que este valor no se conoce antes de realizar el experimento. Por tanto, realizaremos una regresión para calcularlo y junto al notebook de la clasificación binaria podremos decir lo siguiente:

1- Predeciremos entre kaon y pion

2- Calcularemos el valor de la variable ``trueE``

3- Así podremos decir: 'Esto es un kaon/pion y lo hace con la siguiente energía'

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Cambiar el directorio de trabajo
ruta_nueva = 'C:/Users/ruben.morillas/Desktop/aaa/'  # Reemplaza 'TuUsuario' con tu nombre de usuario de Windows
os.chdir(ruta_nueva)

# Mostrar el directorio actual para confirmar el cambio
directorio_actual = os.getcwd()
print("Directorio actual:", directorio_actual)

Directorio actual: C:\Users\ruben.morillas\Desktop\aaa


In [4]:
# Leemos los datos de train
path_actual = os.getcwd()
subdirectorio = 'datas'
file_train = 'df_train.csv'
path_train = os.path.join(path_actual, subdirectorio, file_train)
df_train = pd.read_csv(path_train, index_col=None)

In [5]:
# Leemos los datos de validacion
file_valid = 'df_valid.csv'
path_valid = os.path.join(path_actual,subdirectorio, file_valid)
df_valid = pd.read_csv(path_valid, index_col=None)

In [6]:
df_train

,eventID,PDGcode,trueE,hitX,hitY,hitZ,hitTime,hitInteg
0,1,211,0.381965,13.5922,2.85275,-18.062,1872.38,1655.0600
1,1,211,0.381965,13.8270,2.79629,-17.893,1878.77,340.1960
2,1,211,0.381965,13.6746,2.74496,-17.677,1876.82,4047.6200
3,1,211,0.381965,13.5436,2.71796,-17.580,1878.30,1234.5500
4,1,211,0.381965,13.3256,2.72527,-17.542,1868.97,1394.2800
...,...,...,...,...,...,...,...,...
11612552,19999,321,0.511779,-26.7600,9.41895,45.739,1362.47,1825.6700
11612553,19999,321,0.511779,-26.8500,9.43314,45.757,1358.16,1862.7900
11612554,19999,321,0.511779,-26.8780,9.51059,46.057,1360.96,1586.2400
11612555,19999,321,0.511779,-26.9210,9.52140,46.063,1357.25,388.8380


In [7]:
df_valid

,eventID,PDGcode,trueE,hitX,hitY,hitZ,hitTime,hitInteg
0,0,211,0.258707,-0.0020,-0.173207,0.2,1704.94,318.992
1,0,211,0.258707,0.0000,0.000000,0.5,1704.96,346.856
2,0,211,0.258707,0.0024,0.000000,0.7,1704.99,592.421
3,0,211,0.258707,0.0067,-0.346409,0.7,1698.51,1070.020
4,0,211,0.258707,0.0036,-0.519616,0.8,1698.47,889.101
...,...,...,...,...,...,...,...,...
3267899,19997,321,0.500785,1.9472,-12.124400,4.7,1726.72,357.400
3267900,19997,321,0.500785,1.9682,-12.817200,4.7,1723.61,338.582
3267901,19997,321,0.500785,2.0054,-12.470800,4.7,1727.47,333.317
3267902,19997,321,0.500785,1.6111,-13.683200,4.8,1719.04,368.375


## 0.- Tipos de modelos para el entrenamiento

*   **Regresión Lineal**: Modelo básico que intenta predecir una variable objetivo mediante una combinación lineal de las variables independientes
*   **Regresion Ridge**: Extiende la regresión lineal añadiendo una penalización L2 (la suma de los cuadrados de los coeficientes) a la función de perdida
*   **Regresión Lasso**: Similar a Ridge, pero añade una penalización L1 (la suma de los valores absolutos de los coeficientes),  lo que resulta en modelos con menos coeficientes no cero.
*   **Elastic Net**: Combina las penalizaciones de Ridge y Lasso, útil cuando hay múltiples características que están correlacionadas.
*   **Arboles de decisión para Regresión**: Utiliza un modelo de árbol de decisión para capturar relaciones no lineales en los datos
*   **Random Forest para Regresión**: Un ensamble de árboles de decisión, típicamente más robusto y preciso que un solo árbol.
*   **Gradient Boosting para Regresión**: Construye un modelo aditivo de manera progresiva y es efectivo para modelos complejos.
*   **SVR**: Extensión de las máquinas de soporte vectorial para regresión, intenta ajustar el error dentro de un umbral determinado
*   **K-Nearest Neighbors Regressor**: Predice el valor de la variable dependiente usando el promedio o la mediana de los k vecinos más cercanos
*   **Regresión Bayesiana Ridge**: Variante bayesiana de la regresión Ridge, proporciona una estimación probabilística de los parámetros del modelo
*   **Regresión de Vectores de Soporte Lineal**: Una versión más rápida de SVR para grandes conjuntos de datos con una relación lineal
*   **Stochastic Gradient Descent for Regression**: Utiliza el descenso de gradiente estocástico para encontrar los coeficientes, útil para cojuntos de datos grandes

## 1.- Entrenamiento de distintos modelos para la regresión

### 1.1.- Entrenamiento con el desconocimiento del kaon o pion

Aqui le pasaremos al modelo todos los datos que le pasabamos al problema de la clasificación binaria y el modelo no conocerá si es un kaon o un pion

In [8]:
def filtrado_datos_energia(df, N):
    kaones = []
    labels = []
    valores_trueE = []  # Usamos un conjunto para almacenar valores únicos de trueE

    df_sorted = df.sort_values(by=['eventID', 'hitTime'], ascending=[True, False])

    for eventID, grupo in df_sorted.groupby('eventID'):
        pdgCodes = grupo['PDGcode'].unique()

        for pdgCode in pdgCodes:
            grupo_filtrado = grupo[grupo['PDGcode'] == pdgCode]
            grupo_ordenado = grupo_filtrado.head(N)

            # Obtener el valor de 'trueE' para el evento actual
            trueE_value = grupo_filtrado['trueE'].iloc[0]

            # Almacenar el valor de trueE en el conjunto
            valores_trueE.append(trueE_value)

            # Inicializar arrays para el padding
            hitX_padded = np.zeros(N)
            hitY_padded = np.zeros(N)
            hitZ_padded = np.zeros(N)
            hitInteg_padded = np.zeros(N)

            # Separar y aplicar padding a los valores de hitX, hitY, hitZ, hitInteg
            hitX_padded[:len(grupo_ordenado['hitX'])] = grupo_ordenado['hitX']
            hitY_padded[:len(grupo_ordenado['hitY'])] = grupo_ordenado['hitY']
            hitZ_padded[:len(grupo_ordenado['hitZ'])] = grupo_ordenado['hitZ']
            hitInteg_padded[:len(grupo_ordenado['hitInteg'])] = grupo_ordenado['hitInteg']

            # Concatenar los valores ya con el padding aplicado, añadiendo 'trueE' al principio
            hit_values_reorganized = np.concatenate([[trueE_value], hitX_padded, hitY_padded, hitZ_padded, hitInteg_padded])

            kaones.append(hit_values_reorganized)

            # Modificar las etiquetas de 211 a 0 y de 321 a 1
            if pdgCode == 211:
                labels.append(0)
            elif pdgCode == 321:
                labels.append(1)

    return np.array(kaones), np.array(labels), np.array(list(valores_trueE))

def filtrado_datos(df, N):
    kaones = []
    labels = []

    df_sorted = df.sort_values(by=['eventID', 'hitTime'], ascending=[True, False])

    for eventID, grupo in df_sorted.groupby('eventID'):
        pdgCodes = grupo['PDGcode'].unique()

        for pdgCode in pdgCodes:
            grupo_filtrado = grupo[grupo['PDGcode'] == pdgCode]
            grupo_ordenado = grupo_filtrado.head(N)

            # Inicializar arrays para el padding
            hitX_padded = np.zeros(N)
            hitY_padded = np.zeros(N)
            hitZ_padded = np.zeros(N)
            hitInteg_padded = np.zeros(N)

            # Separar y aplicar padding a los valores de hitX, hitY, hitZ, hitInteg
            hitX_padded[:len(grupo_ordenado['hitX'])] = grupo_ordenado['hitX']
            hitY_padded[:len(grupo_ordenado['hitY'])] = grupo_ordenado['hitY']
            hitZ_padded[:len(grupo_ordenado['hitZ'])] = grupo_ordenado['hitZ']
            hitInteg_padded[:len(grupo_ordenado['hitInteg'])] = grupo_ordenado['hitInteg']

            # Concatenar los valores ya con el padding aplicado
            hit_values_reorganized = np.concatenate([hitX_padded, hitY_padded, hitZ_padded, hitInteg_padded])

            kaones.append(hit_values_reorganized)

            # Modificar las etiquetas de 211 a 0 y de 321 a 1
            if pdgCode == 211:
                labels.append(0)
            elif pdgCode == 321:
                labels.append(1)

    return np.array(kaones), np.array(labels)

In [9]:
N=760
X_train, etiqueta_particula_train, trueE_train = filtrado_datos_energia(df_train, N)
X_train = X_train[:, 1:]

In [10]:
X_valid, etiqueta_particula_valid, trueE_valid = filtrado_datos_energia(df_valid, N)
X_valid = X_valid[:, 1:]


#### funcion para calcular los errores

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def scores_errors(etiquetas_reales, etiquetas_predecidas):
    mse = mean_squared_error(etiquetas_reales, etiquetas_predecidas)
    mae = mean_absolute_error(etiquetas_reales, etiquetas_predecidas)
    r2 = r2_score(etiquetas_reales, etiquetas_predecidas)
    return mse, mae, r2


#### Linear Regresion

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV


param_grid_linear_regresion={
    'fit_intercept' : [True, False],
    'copy_X' : [True, False],
    'positive' : [True,False]
}

grid_search = GridSearchCV(estimator=LinearRegression(), param_grid=param_grid_linear_regresion)
grid_search.fit(X_train, trueE_train)
print(grid_search.best_params_)


In [11]:
model_linear_regresion = LinearRegression(fit_intercept=True, copy_X=True, positive=False).fit(X_train, trueE_train)
y_pred = model_linear_regresion.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

Mean Squared Error: 0.011382157128317126
Mean Abosolute Error: 0.08318043902206823
R2 score: 0.622435218114979


In [12]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.44959877305098006


#### Ridge

In [13]:
from sklearn.linear_model import Ridge

model_ridge = Ridge().fit(X_train, trueE_train)
y_pred = model_ridge.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)


Mean Squared Error: 0.011382074880462757
Mean Abosolute Error: 0.08318026284116473
R2 score: 0.6224379464109291


In [14]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.44959783084798627


#### Lasso

In [15]:
from sklearn.linear_model import Lasso

model_lasso = Lasso().fit(X_train, trueE_train)
y_pred = model_lasso.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)


Mean Squared Error: 0.011475669983184857
Mean Abosolute Error: 0.08505781171050696
R2 score: 0.6193332436602652


In [16]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.43483632373270803


#### Elastic Net

In [17]:
from sklearn.linear_model import ElasticNet

model_elastic_net = ElasticNet().fit(X_train, trueE_train)
y_pred = model_elastic_net.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

Mean Squared Error: 0.01146273264488675
Mean Abosolute Error: 0.08470316124633308
R2 score: 0.6197623963470212


In [18]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.4334752091236372


#### Polinomial Features

In [19]:
from sklearn.tree import DecisionTreeRegressor

model_decision_tree = DecisionTreeRegressor().fit(X_train, trueE_train)
y_pred = model_decision_tree.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)


Mean Squared Error: 0.014445068484888354
Mean Abosolute Error: 0.07468906754530479
R2 score: 0.520833435145396


In [20]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.493809


#### Suport Vector Regression

In [40]:
from sklearn.neighbors import KNeighborsRegressor

model_KNN = KNeighborsRegressor().fit(X_train, trueE_train)
y_pred = model_KNN.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

Mean Squared Error: 0.01702453199406248
Mean Abosolute Error: 0.10026350987200608
R2 score: 0.43526840856543925


In [41]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.27581500000000003


### Bayesian Ridge Regression

In [42]:
from sklearn.linear_model import BayesianRidge

model_bayesian_ridge = BayesianRidge().fit(X_train, trueE_train)
y_pred = model_bayesian_ridge.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

Mean Squared Error: 0.009276232145118208
Mean Abosolute Error: 0.07605977736937516
R2 score: 0.6922921967161237


In [43]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.4247767377111192


#### Regression de vectores de Soporte Vital

In [44]:
from sklearn.svm import LinearSVR

model_linearSVR = LinearSVR().fit(X_train, trueE_train)
y_pred = model_linearSVR.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

C:\Users\ruben.morillas\AppData\Local\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Mean Squared Error: 0.0364748629387129
Mean Abosolute Error: 0.14571145022913656
R2 score: -0.2099309045277018


C:\Users\ruben.morillas\AppData\Local\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [45]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.5403938243362615


#### Stochastic Gradiente Descent for Regresion

In [46]:
from sklearn.linear_model import SGDRegressor

model_SGD = SGDRegressor().fit(X_train, trueE_train)
y_pred = model_SGD.predict(X_valid)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

Mean Squared Error: 3.2055295352001235e+33
Mean Abosolute Error: 4.190716171170152e+16
R2 score: -1.0633266138742642e+35


In [47]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido -3.818594678633532e+16


### 1.2.- Entrenamiento conociendo si la particula es kaon/pion, por prediccion

Para ello deberemos pedirle a nuestro modelo de clasificación binaria que nos prediga si el evento corresponde al de una particula kaon o al de una particula pion

In [48]:
import joblib
## Leemos el modelo

model_binary = joblib.load('model_binary.pkl')

In [54]:
predicciones_particula_train = model_binary.predict(X_train)
predicciones_particula_valid = model_binary.predict(X_valid)

In [55]:
# Asegúrate de que predicciones_particula_train sea una matriz unidimensional
predicciones_particula_train = np.reshape(predicciones_particula_train, (-1, 1))
predicciones_particula_valid = np.reshape(predicciones_particula_valid, (-1, 1))

# Añadir la columna de predicciones a X_train
X_train_con_predicciones = np.column_stack((X_train, predicciones_particula_train))
X_valid_con_predicciones = np.column_stack((X_valid, predicciones_particula_valid))


Ahora probaremos los dos modelos anteriores que mejores resultados hayan dado

#### Bayessian Ridge Regression

In [56]:
from sklearn.linear_model import BayesianRidge

model_bayesian_ridge = BayesianRidge().fit(X_train_con_predicciones, trueE_train)
y_pred = model_bayesian_ridge.predict(X_valid_con_predicciones)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

Mean Squared Error: 0.009273509767388678
Mean Abosolute Error: 0.07604590485407857
R2 score: 0.6923825024413102


In [57]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.4248211114884911


#### Polinomial Features

In [58]:
from sklearn.tree import DecisionTreeRegressor

model_decision_tree = DecisionTreeRegressor().fit(X_train_con_predicciones, trueE_train)
y_pred = model_decision_tree.predict(X_valid_con_predicciones)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

Mean Squared Error: 0.0111648623090275
Mean Abosolute Error: 0.06362626308452668
R2 score: 0.6296432429317972


In [59]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.49928


#### Regresion Lineal

In [60]:
from sklearn.linear_model import LinearRegression

model_linear_regresion = LinearRegression(fit_intercept=True, copy_X=True, positive=False).fit(X_train_con_predicciones, trueE_train)
y_pred = model_linear_regresion.predict(X_valid_con_predicciones)

mse, mae, r2 = scores_errors(trueE_valid, y_pred)
print("Mean Squared Error:", mse)
print("Mean Abosolute Error:", mae)
print("R2 score:", r2)

Mean Squared Error: 0.008125892684760853
Mean Abosolute Error: 0.06580804547944726
R2 score: 0.7304508394538007


In [61]:
print('Predicción aislada')
print('Valor real', trueE_valid[100])
print('Valor prededecido', y_pred[100])

Predicción aislada
Valor real 0.496129
Valor prededecido 0.4874450710614595
